In [1]:
# import module list is independent on respective file.
import yfinance as yf
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

from datetime import datetime as dt
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,  MinMaxScaler
from scipy.stats import mstats
import japanize_matplotlib
from tqdm.notebook import tqdm



NameError: name 'skip_to_date' is not defined

In [3]:
skip_to_date = '2005-10-31'
bond1 = pd.read_csv('data/MSPD1.csv', index_col=0, parse_dates=True).loc[skip_to_date:, 'Total Public Debt Outstanding (in Millions)']
bond2 = pd.read_csv('data/MSPD1.csv', index_col=0, parse_dates=True).loc[skip_to_date:, 'Intragovernmental Holdings (in Millions)']

In [20]:
# load correct trading day format

csv_file_path = './data/Train_data_all.csv' 
Train_data = pd.read_csv(csv_file_path, index_col=0, thousands=",", parse_dates=True) 
# read different lenghth for cutting
skip_to_date = "2005-12-01" 


# input the ticker
ticker = "IYR" #Real Estate SPDR  



#     Load saved share price file
#file_path = '~/data/' #ダウンロードした人用
file_path = './data/' #自分用
file_name = ticker
file_extension = '.csv'
price_list = pd.read_csv(file_path + file_name + file_extension,thousands=",", index_col=0, parse_dates=True)           

In [21]:
Train_data = pd.read_csv('./data/Train_data_all.csv' , index_col=0, parse_dates=True)
Train_data

,Monthly Open,futures,Outlays - Receipts,US Bond,Outlays,Receipts,Government Account Series,State and Local Government Series,Treasury Inflation-Protected Securities,Principal Amount Balance,...,Net Unamortized Amount Increase,Principal Amount Decrease,Net Unamortized Amount Decrease,Accrued Interest Payable Amount,Accrued Interest Increase,Accrued Interest decrease,bond1,bond2,Floating_Rate,saving_securities
Date,,,,,,,,,,,,,,,,,,,,,
2001-01-31,2001/1/1,105.22,-76379,5.179,142836,219215,2348243.00,147898,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-02-28,2001/2/1,106.33,48168,4.908,158649,110481,2354000.00,149904,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-03-31,2001/3/1,106.20,50662,4.915,180736,130074,2360298.00,152887,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-04-30,2001/4/1,103.78,-189796,5.338,141999,331796,2385627.00,154746,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2001-05-31,2001/5/1,103.69,27919,5.413,153508,125590,2401909.00,155955,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-30,2023/6/1,112.88,227768,3.819,646085,418317,7150742.00,90481,1933629.0,25461998.0,...,-25961.0,1106743.0,-17725.0,98908.0,49285.0,27559.0,4170502.205,6805.3735,586359.7624,178253.3667
2023-07-31,2023/7/1,111.97,220782,3.959,496942,276160,7178591.00,88043,1901965.0,25708799.0,...,-23047.0,1511497.0,-20551.0,111448.0,43746.0,31206.0,4200350.812,6686.2202,527566.6634,177725.0661
2023-08-31,2023/8/1,111.03,-89255,4.093,193874,283130,7148949.24,84665,19162527.0,26044051.0,...,-28679.0,1996860.0,-22111.0,81711.0,45191.0,74928.0,4226927.268,6729.4072,551555.9597,176640.5119


In [22]:
# use correct format's index to retrieve the trading data
post_fundamental = Train_data[skip_to_date:]  

idx = np.searchsorted(price_list.index, post_fundamental.index)
#documents = _list.iloc[idx]
fold_index= 3

In [23]:
def folding_train_data(df, index2):
    try:
        if "index" in df.columns.to_list():
            name = "index"
        else:
            df = df.reset_index(drop=False)
            name = "Date"
        df = df[df.index % index2==0]
        
        
        
    except Exception as e:
        raise e
    else:
        
        df.set_index(name,drop=True, inplace=True)
    
    return df 




In [27]:
fold_index= 3
#post_fundamental.rename_axis('Date', inplace=True)
post_fundamental = post_fundamental.loc[skip_to_date:'2023-09-30', :].asfreq(str(fold_index)+'M')

In [28]:
post_fundamental

,Monthly Open,futures,Outlays - Receipts,US Bond,Outlays,Receipts,Government Account Series,State and Local Government Series,Treasury Inflation-Protected Securities,Principal Amount Balance,...,Net Unamortized Amount Increase,Principal Amount Decrease,Net Unamortized Amount Decrease,Accrued Interest Payable Amount,Accrued Interest Increase,Accrued Interest decrease,bond1,bond2,Floating_Rate,saving_securities
Date,,,,,,,,,,,,,,,,,,,,,
2005-12-31,2005/12/1,109.41,-10967,4.395,230916,241883,3506577.0,235566,328678.0,4714821.0,...,-3880.0,400621.0,-3424.0,40196.0,14014.0,3535.0,5.166210e+05,196.4280,NaN,205162.4211
2006-03-31,2006/3/1,106.39,85281,4.853,249843,164563,3551185.0,234844,347899.0,4872485.0,...,-5059.0,403915.0,-3969.0,36621.0,16073.0,6552.0,5.267037e+05,202.2760,NaN,205975.6350
2006-06-30,2006/6/1,104.86,-20517,5.138,243838,264355,3680235.0,242022,372757.0,4797205.0,...,-4310.0,415368.0,-4163.0,40786.0,16903.0,8339.0,5.231570e+05,233.3250,NaN,205227.2227
2006-09-30,2006/9/1,108.06,-56167,4.633,227131,283298,3722747.0,238835,395638.0,4843121.0,...,NaN,NaN,NaN,41119.0,NaN,NaN,5.347099e+05,236.5710,NaN,203656.3217
2006-12-31,2006/12/1,107.47,-41961,4.710,218007,259969,3839348.0,257634,411193.0,4901047.0,...,-3657.0,342301.0,-4421.0,45517.0,12502.0,1018.0,5.306820e+05,134.4030,NaN,202433.3057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-30,2022/9/1,112.38,429776,3.804,917489,487723,6929803.0,109236,1840532.0,24299193.0,...,NaN,NaN,NaN,81183.0,NaN,NaN,3.874352e+06,6680.4895,625977.1773,166292.4312
2022-12-31,2022/12/1,112.89,85001,3.879,539943,454942,7179347.0,99898,1908072.0,24517593.0,...,-13193.0,1305688.0,-12171.0,99513.0,41677.0,10254.0,3.959874e+06,7215.6275,617204.1917,173499.8057
2023-03-31,2023/3/1,115.61,378076,3.494,691317,313240,6772639.0,98170,1905608.0,24693515.0,...,-18797.0,1507571.0,-15961.0,92773.0,49846.0,27731.0,4.063694e+06,6972.9905,601407.1949,177841.0264


In [47]:
DS = post_fundamental.loc[:,'Outlays - Receipts'].values

US_Bond = post_fundamental.loc[:,'US Bond'].values
Futures = post_fundamental.loc[:,'futures'].values

Receipts = post_fundamental.loc[:,'Receipts'].values

Outlays = post_fundamental.loc[:,'Outlays'].values

SLGS = post_fundamental.loc[:,'State and Local Government Series'].values

GAS = post_fundamental.loc[:,'Government Account Series'].values
TIPS = post_fundamental.loc[:,'Treasury Inflation-Protected Securities'].values

PAB = post_fundamental.loc[:,'Principal Amount  Balance'].values
AIPB = post_fundamental.loc[:,'Accrued Interest Payable Amount'].values

NUAB = post_fundamental.loc[:,'Net Unamortized Amount Balance'].values

Bill = post_fundamental.loc[:,'Bill(Net Sales Amount)'].values
Note = post_fundamental.loc[:,'Note(Net Sales Amount)'].values
Bond = post_fundamental.loc[:,'Bond(Net Sales Amount)'].values



SBI = post_fundamental.loc[:,'Saving Bond typeI'].values
SBEE = post_fundamental.loc[:,'Saving Bond typeEE'].values

PAI = post_fundamental.loc[:,'Principal Amount Increase'].values
AII = post_fundamental.loc[:,'Accrued Interest Increase'].values
NUAI = post_fundamental.loc[:,'Net Unamortized Amount Increase'].values

PAD = post_fundamental.loc[:,'Principal Amount Decrease'].values
AID = post_fundamental.loc[:,'Accrued Interest decrease'].values
NUAD = post_fundamental.loc[:,'Net Unamortized Amount Decrease'].values

bond1 = post_fundamental.loc[:,'bond1'].values
bond2 = post_fundamental.loc[:,'bond2'].values
SS = post_fundamental.loc[:,'saving_securities'].values

In [13]:
file_name

'IYR'

In [56]:
# Goal is to anticipate the sign of future rate change from the financial data of the current monthly.
# If the future changes is + , we assign 1, otherwise 0,  to Future change value of the current quarter


# Returned Sales Amount.

# predict the FFO direction

data_2 = []
data_2 = pd.DataFrame(data_2)

# 2023 09 DS, Receipts, GAS, NUA
# 2023 09 DS Receipts Bond NUAB
data_2['DS'] = DS

data_2['SLGS'] = SLGS
data_2["GAS"] = GAS
data_2['Receipts'] = Receipts

# Principal Amount Balance
data_2['PAB'] = PAB
data_2['AIPB'] = AIPB


data_2['Bill'] = Bill
data_2['Note'] = Note
data_2['Bond'] = Bond
'''
data_2['Bill2'] = Bill2
data_2['Note2'] = Note2
data_2['Bond2'] = Bond2
'''
data_2['NUAB'] = NUAB
data_2['SBI'] = SBI
data_2['SBEE'] = SBEE
data_2['AII'] = AII
data_2['PAI'] = PAI
data_2['NUAI'] = NUAI

data_2['PAD'] = PAD 
data_2['AID'] = AID
data_2['NUAD'] = NUAD

data_2['bond1'] = bond1
data_2['bond2'] = bond2
data_2['SS'] = SS

In [15]:
bond1 PAB GAS AIPA

SyntaxError: invalid syntax (3925843809.py, line 1)

In [57]:
USBond_change = np.where(US_Bond[1:] > US_Bond[0:-1], 1, 0)

data_2['class'] = np.append(USBond_change,1) 
data_2

,DS,SLGS,GAS,Receipts,PAB,AIPB,Bill,Note,Bond,NUAB,...,AII,PAI,NUAI,PAD,AID,NUAD,bond1,bond2,SS,class
0,-10967,235566,3506577.0,241883,4714821.0,40196.0,5.485360e+08,3831000.0,NaN,-36826.0,...,14014.0,406039.0,-3880.0,400621.0,3535.0,-3424.0,5.166210e+05,196.4280,205162.4211,1
1,85281,234844,3551185.0,164563,4872485.0,36621.0,1.238934e+09,28038000.0,NaN,-38960.0,...,16073.0,505283.0,-5059.0,403915.0,6552.0,-3969.0,5.267037e+05,202.2760,205975.6350,1
2,-20517,242022,3680235.0,264355,4797205.0,40786.0,1.553640e+09,51579000.0,NaN,-38145.0,...,16903.0,405056.0,-4310.0,415368.0,8339.0,-4163.0,5.231570e+05,233.3250,205227.2227,0
3,-56167,238835,3722747.0,283298,4843121.0,41119.0,1.356214e+09,1440000.0,NaN,-40165.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5.347099e+05,236.5710,203656.3217,1
4,-41961,257634,3839348.0,259969,4901047.0,45517.0,1.344084e+09,742000.0,NaN,-39623.0,...,12502.0,313781.0,-3657.0,342301.0,1018.0,-4421.0,5.306820e+05,134.4030,202433.3057,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,429776,109236,6929803.0,487723,24299193.0,81183.0,7.744515e+09,839242100.0,7661600.0,-71681.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.874352e+06,6680.4895,166292.4312,1
68,85001,99898,7179347.0,454942,24517593.0,99513.0,1.105938e+10,186215100.0,1678700.0,-89151.0,...,41677.0,1269824.0,-13193.0,1305688.0,10254.0,-12171.0,3.959874e+06,7215.6275,173499.8057,0
69,378076,98170,6772639.0,313240,24693515.0,92773.0,1.657816e+10,451514900.0,21094800.0,-95111.0,...,49846.0,1591845.0,-18797.0,1507571.0,27731.0,-15961.0,4.063694e+06,6972.9905,177841.0264,1
70,227768,90481,7150742.0,418317,25461998.0,98908.0,1.412897e+10,299794100.0,11658600.0,-108085.0,...,49285.0,1938055.0,-25961.0,1106743.0,27559.0,-17725.0,4.170502e+06,6805.3735,178253.3667,1


In [58]:
data_2.index = post_fundamental.index

In [59]:
data_2['month'] = data_2.index.month
data_2.query('month==12 | month== 6')

,DS,SLGS,GAS,Receipts,PAB,AIPB,Bill,Note,Bond,NUAB,...,PAI,NUAI,PAD,AID,NUAD,bond1,bond2,SS,class,month
Date,,,,,,,,,,,,,,,,,,,,,
2005-12-31,-10967,235566,3506577.0,241883,4714821.0,40196.0,5.485360e+08,3831000.0,NaN,-36826.0,...,406039.0,-3880.0,400621.0,3535.0,-3424.0,5.166210e+05,196.428000,205162.4211,1,12
2006-06-30,-20517,242022,3680235.0,264355,4797205.0,40786.0,1.553640e+09,51579000.0,NaN,-38145.0,...,405056.0,-4310.0,415368.0,8339.0,-4163.0,5.231570e+05,233.325000,205227.2227,0,6
2006-12-31,-41961,257634,3839348.0,259969,4901047.0,45517.0,1.344084e+09,742000.0,NaN,-39623.0,...,313781.0,-3657.0,342301.0,1018.0,-4421.0,5.306820e+05,134.403000,202433.3057,0,12
2007-06-30,-27481,301624,3989271.0,276517,4943092.0,47602.0,1.575807e+09,3641000.0,NaN,-39175.0,...,294177.0,-3767.0,331956.0,6388.0,-3870.0,5.472553e+05,123.893000,198647.7968,0,6
2007-12-31,-48261,293239,4164266.0,276982,5136303.0,45199.0,1.132988e+09,20767000.0,NaN,-37237.0,...,370756.0,-2503.0,381032.0,8233.0,-3700.0,5.585378e+05,90.923000,196452.3789,0,12
2008-06-30,-33547,275203,4288079.0,259912,5285064.0,44112.0,1.105164e+09,48605300.0,NaN,-34535.0,...,443279.0,-2359.0,436914.0,14027.0,-1946.0,5.810589e+05,64.000000,195002.5403,0,6
2008-12-31,51754,249318,4421658.0,237786,6369319.0,44443.0,8.684304e+08,31693500.0,NaN,-34119.0,...,574032.0,648.0,601304.0,5768.0,-2149.0,5.946411e+05,2774.061773,194074.5565,1,12
2009-06-30,94332,232965,4468613.0,215340,7174912.0,44498.0,8.660976e+08,47954100.0,4673000.0,-33257.0,...,788076.0,-3217.0,632486.0,11955.0,-1019.0,6.436603e+05,1609.950800,193591.5504,0,6
2009-12-31,91410,214138,4597132.0,218919,7811009.0,47316.0,9.058589e+08,29596600.0,2365600.0,-32129.0,...,731419.0,-1201.0,632797.0,7736.0,-661.0,7.179312e+05,3259.052000,191297.8089,0,12


In [84]:
data_2 = data_2.query('month==12 | month== 6')
train_len = 7


#namelist = [['Receipts', 'AID', 'NUAI', 'AIPA']]
#namelist = [['PAD', 'NUAI', 'NUAD', 'NUAB']]
namelist = ['Receipts', 'AID', 'NUAI', 'AIPB']

for j in range(0, 1):
    value2 = 0
    num1 = 0
    df = data_2.drop(columns=['class'])[namelist]

    
    for num in range(5, 10, 5):
        # 欠損値を列の1つ手前の値で埋める
        df = df.dropna(how='any').dropna(how='all', axis=0)
        df = df.fillna(method='ffill')
        X = df.values

        


        y = data_2['class'].values


        y_test = []
        rate_prediction = []

        result = []
        result = pd.DataFrame(result)
        for i in range(len(y)-train_len+1):
            X_train = X[0:train_len+i-1,:]    # train until previous period, cuz y is forecasting next period
            y_train = y[0:train_len+i-1]
            X_predict = X[0:train_len+i,:] # use most recent reporting period for forecasting

            # Winsorize top 1% and bottom 1% of points.
            # Apply on X_train and X_test separately
            X_train = mstats.winsorize(X_train, limits = [0.01, 0.01])
            X_predict = mstats.winsorize(X_predict, limits = [0.01, 0.01])

            sc = StandardScaler()
            # Fit to training data and then transform it
            X_train = sc.fit_transform(X_train) # SVM is unsuitable to raw larger-scale number, However, Standard saves this.
            X_predict = sc.transform(X_predict) 

            y_test.append(y[train_len+i-1]) # 6, 7, 8, 9, 10, 11.......

            # Initialize svm, rbf is a default kernel
            classifier_rbf = SVC(C=5, kernel = 'rbf', gamma = 'auto', random_state=0)


            # Fit the model on training data

            # Make a prediction on testing data
            try:
                classifier_rbf.fit(X_train, y_train)
                y_pred_rbf = classifier_rbf.predict(X_predict[train_len -1+i,:].reshape(1,-1))
            except:
                pass


            rate_prediction = np.append(rate_prediction,y_pred_rbf)





        result['predict'] = rate_prediction


        from sklearn.metrics import matthews_corrcoef
        value= matthews_corrcoef(y_test, rate_prediction)
        if value < 0.13:
            continue
        elif value >= value2:
            value2 = value
            num1 = num
    if value2 < 0.13:
        continue
    print('corr:', value2, df.columns, num1)


    result.index = [i+timedelta(days=1) for i in data_2.index[train_len-1:]]






In [ ]:
corr: 0.1336306209562122 Index(['PAI', 'bond1', 'NUAI', 'SBI'], dtype='object') 40
corr: 0.13608276348795434 Index(['PAD', 'bond1', 'NUAB', 'NUAI'], dtype='object') 15


In [ ]:
1126 

〇corr: 0.15498260496951669 Index(['NUAB', 'DS', 'Receipts', 'PAB'], dtype='object') 45
    
    ['NUAD', 'AII', 'PAD', 'NUAI']


In [81]:
# Export data to CSV file
export = True
file_path = './prediction/'
file_name = "US3mo1218Test3"
file_extension = '.csv'

#     new_copy.to_csv(file_path + file_name + file_extension)
if export:
    data = result
    data.to_csv(file_path + file_name + file_extension)

In [ ]:
df